In [1]:
import pandas as pd
import numpy as np
from scipy import sparse

from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, StandardScaler, LabelEncoder
#from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin

from datetime import datetime
from datetime import date

import re
#import os, re, sys
import pickle

In [3]:
from xgboost import XGBClassifier
import xgboost as xgb

In [4]:
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet')

In [5]:
from nltk.corpus import stopwords                   #Stopwords corpus
from nltk.tokenize import word_tokenize

In [6]:
import textblob
from textblob import TextBlob

In [7]:
from string import punctuation
##  from spellchecker import SpellChecker

In [8]:
from transforms_dev import *

In [9]:
import requests,json

import csv
from datetime import datetime

In [10]:
import shap

ModuleNotFoundError: No module named 'shap'

### read data

In [11]:
df_tmp = pd.read_csv('C:/Users/lguerra5/Documents/FNOP/data/Traindata_pilot_sample_v0.csv', engine='python')

In [12]:
len(set(df_tmp['target']))

121

In [13]:
df_tmp.columns

Index(['clm_key', 'cust_txt', 'mdl_yr', 'prt_num_causl_base_cd', 'tis_wsd',
       'milge', 'veh_line_cd', 'eng_cd', 'cust_conc_cd', 'part_desc',
       'target'],
      dtype='object')

## define predictors without using customer_txt

### one-hot encoding input variables

In [14]:
df_X = df_tmp[['mdl_yr','tis_wsd','milge', 'veh_line_cd', 'eng_cd']]

In [15]:
for f in df_X.columns:
    print(f, df_X[f].dtype)

mdl_yr int64
tis_wsd int64
milge float64
veh_line_cd object
eng_cd object


In [16]:
df_X['mdl_yr'] = df_X['mdl_yr'].astype('object')

C:\Users\lguerra5\AppData\Local\Continuum\anaconda3\envs\fnop\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
for f in df_X.columns:
    print(f, df_X[f].dtype)

mdl_yr object
tis_wsd int64
milge float64
veh_line_cd object
eng_cd object


In [18]:
## convert to one-hot encoding
df_X_dummy = pd.get_dummies(df_X)

In [19]:
df_X_dummy.shape

(358000, 162)

In [20]:
df_X_dummy.head(5)

tis_wsd    milge  mdl_yr_2011  mdl_yr_2012  mdl_yr_2013  mdl_yr_2014  \
0       13  14776.0            1            0            0            0   
1       15  15146.0            1            0            0            0   
2       13  11198.0            1            0            0            0   
3        0    731.0            1            0            0            0   
4       22   4471.0            1            0            0            0   

   mdl_yr_2015  mdl_yr_2016  mdl_yr_2017  mdl_yr_2018  ...  eng_cd_T/UR  \
0            0            0            0            0  ...            0   
1            0            0            0            0  ...            0   
2            0            0            0            0  ...            0   
3            0            0            0            0  ...            0   
4            0            0            0            0  ...            0   

   eng_cd_T/VN  eng_cd_T/VZ  eng_cd_T/WA  eng_cd_T/WQ  eng_cd_T/WX  \
0            0            0            0            0            0   
1            0            0            0            0            0   
2            0            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   eng_cd_T/YE  eng_cd_T/YF  eng_cd_T/YG  eng_cd_T/YH  
0            0            0            0            0  
1            0            0            0            0  
2            0            0            0            0  
3            0            0            0            0  
4            0            0            0            0  

[5 rows x 162 columns]

In [22]:
df_X_dummy.columns

Index(['tis_wsd', 'milge', 'mdl_yr_2011', 'mdl_yr_2012', 'mdl_yr_2013',
       'mdl_yr_2014', 'mdl_yr_2015', 'mdl_yr_2016', 'mdl_yr_2017',
       'mdl_yr_2018',
       ...
       'eng_cd_T/UR', 'eng_cd_T/VN', 'eng_cd_T/VZ', 'eng_cd_T/WA',
       'eng_cd_T/WQ', 'eng_cd_T/WX', 'eng_cd_T/YE', 'eng_cd_T/YF',
       'eng_cd_T/YG', 'eng_cd_T/YH'],
      dtype='object', length=162)


### label encoding target variable

In [21]:
le = LabelEncoder()
le.fit(df_tmp['target'])
label_encoded_y = le.transform(df_tmp['target'])

#### XGBoost training

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_X_dummy, label_encoded_y, test_size = 0.2, random_state = 11)

In [23]:
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

In [24]:
params = {
    "objective": "multi:softprob",
    "num_class": 121,
    ##"eval_metric": "auc",
    "booster": "gbtree",
    "tree_method": "exact",
    "eta": 0.1,
    "max_depth": 4,
    "sub_sample": 1,
    "col_sample_bytree": 1,
    "verbosity": 0,    
    "seed": 11    
}

In [25]:
num_rounds = 50
early_stopping_rounds = 10

In [26]:
eval_set = [(dtrain, 'train'), (dtest, 'eval')]

In [27]:
gbm = xgb.train(params, dtrain, num_rounds, evals=eval_set, early_stopping_rounds=early_stopping_rounds)

[0]	train-merror:0.777943	eval-merror:0.779525
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0.764612	eval-merror:0.767053
[2]	train-merror:0.76059	eval-merror:0.761983
[3]	train-merror:0.759155	eval-merror:0.761578
[4]	train-merror:0.756128	eval-merror:0.758952
[5]	train-merror:0.755482	eval-merror:0.75838
[6]	train-merror:0.75514	eval-merror:0.758212
[7]	train-merror:0.753394	eval-merror:0.757039
[8]	train-merror:0.753296	eval-merror:0.756634
[9]	train-merror:0.75286	eval-merror:0.756383
[10]	train-merror:0.751788	eval-merror:0.755754
[11]	train-merror:0.751728	eval-merror:0.755712
[12]	train-merror:0.751477	eval-merror:0.755419
[13]	train-merror:0.750656	eval-merror:0.754483
[14]	train-merror:0.750234	eval-merror:0.754092
[15]	train-merror:0.749759	eval-merror:0.753799
[16]	train-merror:0.749476	eval-merror:0.753757
[17]	train-merror:0.74868	eval-merror:0.752807
[18]

### predict the probability

In [28]:
probs = gbm.predict(xgb.DMatrix(X_test))

In [29]:
probs.shape

(71600, 121)

In [30]:
df_prob = pd.DataFrame(probs, columns=le.classes_)

In [31]:
df_prob.head(10)

10346    10V346     11002      1104      1109      1125      1175  \
0  0.014906  0.000793  0.023393  0.047324  0.001890  0.047339  0.000815   
1  0.004134  0.000597  0.003825  0.025856  0.000414  0.003757  0.000239   
2  0.004996  0.002110  0.005902  0.097931  0.004861  0.007841  0.000845   
3  0.010870  0.000787  0.009066  0.022220  0.001059  0.008025  0.000628   
4  0.014699  0.000658  0.009764  0.023517  0.000957  0.179051  0.000677   
5  0.011374  0.000640  0.009486  0.048049  0.000994  0.052615  0.000657   
6  0.013244  0.001388  0.026288  0.058702  0.001012  0.006949  0.000880   
7  0.011184  0.001448  0.006863  0.171256  0.000873  0.006460  0.000646   
8  0.006368  0.000306  0.005795  0.006086  0.000420  0.015738  0.000315   
9  0.013279  0.001562  0.011075  0.135134  0.001344  0.014290  0.000768   

       1190    11V002     12029  ...     8C607     8C628  99000A25  99286B26  \
0  0.000782  0.000839  0.018554  ...  0.011740  0.000782  0.000779  0.000794   
1  0.000229  0.000661  0.006352  ...  0.003441  0.000229  0.000228  0.000233   
2  0.000811  0.002594  0.047144  ...  0.011918  0.000810  0.000807  0.000823   
3  0.000603  0.000782  0.008349  ...  0.184923  0.000602  0.000600  0.000612   
4  0.000649  0.000664  0.016221  ...  0.008611  0.000649  0.000646  0.000659   
5  0.000631  0.000645  0.014961  ...  0.009467  0.000630  0.000628  0.000640   
6  0.000554  0.005524  0.000928  ...  0.000915  0.000554  0.000629  0.000562   
7  0.000620  0.001386  0.010012  ...  0.010410  0.000620  0.000617  0.000629   
8  0.000302  0.000309  0.005125  ...  0.529984  0.000302  0.000300  0.000306   
9  0.000736  0.000955  0.017686  ...  0.011053  0.000736  0.000733  0.000747   

      9A818     9D653     9F593     9G444    other1    other2  
0  0.000796  0.008830  0.009310  0.015326  0.001967  0.003686  
1  0.000233  0.001665  0.004090  0.004862  0.000429  0.000887  
2  0.000825  0.006121  0.058650  0.035459  0.001874  0.052326  
3  0.000613  0.005131  0.005653  0.014488  0.001127  0.002333  
4  0.000660  0.005269  0.007816  0.024045  0.001500  0.002884  
5  0.000642  0.005080  0.007507  0.018474  0.001170  0.001540  
6  0.000573  0.000574  0.001365  0.001558  0.001420  0.002541  
7  0.000631  0.004681  0.010074  0.018688  0.001366  0.002202  
8  0.000307  0.002547  0.002625  0.008407  0.146646  0.042706  
9  0.000749  0.005347  0.030776  0.016523  0.001887  0.003376  

[10 rows x 121 columns]

In [32]:
## find the top3 parts
k = 3

# GET TOP K PREDICTIONS BY PROB - note these are just index
best_n = np.argsort(probs, axis=1)[:,-k:]

# GET CATEGORY OF PREDICTIONS
preds=[[le.classes_[predicted_cat] for predicted_cat in prediction] for prediction in best_n]

preds=[item[::-1] for item in preds]

df_top3 = pd.DataFrame(preds, columns=("Top1", "Top2", "Top3"))
df_top3.head(10)

Top1    Top2    Top3
0    3504  15K601   19703
1    8501    1104   19972
2    1104  19G490   9F593
3  19E616   8C607    8501
4    1125   18125  15K601
5   19703  15K601  19G490
6   6K682    8501   19860
7    1104  19G490  19E616
8   8C607  other1  other2
9    1104    8501   6K682

In [33]:
### print the actual part from test set
y_test_part = le.inverse_transform(y_test)

### calculate accuracy

In [34]:
N = len(y_test)
n_correct = 0
for i in range(N):
    if (df_top3.iloc[i, 0]==y_test_part[i]) or (df_top3.iloc[i, 1]==y_test_part[i]) or (df_top3.iloc[i, 2]==y_test_part[i]):
        n_correct += 1
        
print(n_correct/N)

0.45090782122905027


### include reliability model scores

In [35]:
reliability_df = pd.read_csv("C:/Users/lguerra5/Documents/FNOP/data/f_rate_WB_118month_poc3.csv", engine='python')

In [36]:
reliability_df.head(5)

prt_num veh_line  Model Year  tis  Avg_Weibull_Failure    f_rate
0   10346     C/AU        2011    1             0.000039  0.000057
1   10346     C/AU        2011    2             0.000071  0.000067
2   10346     C/AU        2011    3             0.000101  0.000096
3   10346     C/AU        2011    4             0.000130  0.000105
4   10346     C/AU        2011    5             0.000158  0.000124

In [37]:
reliability_df.columns = ['prt_num', 'veh_line_cd', 'mdl_yr', 'tis_wsd', 'Avg_Weibull_Failure', 'f_rate']

In [38]:
data_reliab_pivot = reliability_df.pivot_table(index=['veh_line_cd', 'mdl_yr', 'tis_wsd'], columns='prt_num', values=['Avg_Weibull_Failure'], aggfunc='first')
        
reliability_df_t = data_reliab_pivot.fillna(0.0)

In [39]:
reliability_df_t.shape

(24426, 111)

In [40]:
reliability_df_t.head(5)

Avg_Weibull_Failure                             \
prt_num                                  10346 10370         10655 10C679   
veh_line_cd mdl_yr tis_wsd                                                  
C/AU        2011   1                  0.000039   0.0  8.949473e-07    0.0   
                   2                  0.000071   0.0  3.916412e-06    0.0   
                   3                  0.000101   0.0  9.287561e-06    0.0   
                   4                  0.000130   0.0  1.713867e-05    0.0   
                   5                  0.000158   0.0  2.756513e-05    0.0   

                                                                     ...  \
prt_num                    11002      1104 1109 1131 1153      1177  ...   
veh_line_cd mdl_yr tis_wsd                                           ...   
C/AU        2011   1         0.0  0.000008  0.0  0.0  0.0  0.000009  ...   
                   2         0.0  0.000018  0.0  0.0  0.0  0.000030  ...   
                   3         0.0  0.000031  0.0  0.0  0.0  0.000061  ...   
                   4         0.0  0.000044  0.0  0.0  0.0  0.000100  ...   
                   5         0.0  0.000059  0.0  0.0  0.0  0.000148  ...   

                                                                          \
prt_num                         6068   65  67R 7210 7251 7B155   90 90T5   
veh_line_cd mdl_yr tis_wsd                                                 
C/AU        2011   1        0.000010  0.0  0.0  0.0  0.0   0.0  0.0  0.0   
                   2        0.000027  0.0  0.0  0.0  0.0   0.0  0.0  0.0   
                   3        0.000047  0.0  0.0  0.0  0.0   0.0  0.0  0.0   
                   4        0.000071  0.0  0.0  0.0  0.0   0.0  0.0  0.0   
                   5        0.000097  0.0  0.0  0.0  0.0   0.0  0.0  0.0   

                                            
prt_num                    94RH7       96R  
veh_line_cd mdl_yr tis_wsd                  
C/AU        2011   1         0.0  0.000007  
                   2         0.0  0.000038  
                   3         0.0  0.000105  
                   4         0.0  0.000215  
                   5         0.0  0.000375  

[5 rows x 111 columns]

In [41]:
df1 = pd.merge(df_X, reliability_df_t, how='left', on=['veh_line_cd', 'mdl_yr', 'tis_wsd'])

C:\Users\lguerra5\AppData\Local\Continuum\anaconda3\envs\fnop\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [42]:
df1.head(5)

mdl_yr  tis_wsd    milge veh_line_cd eng_cd  (Avg_Weibull_Failure, 10346)  \
0   2011       13  14776.0        C/AU   C/H4                      0.000363   
1   2011       15  15146.0        C/AU   C/H4                      0.000411   
2   2011       13  11198.0        C/AU   C/H4                      0.000363   
3   2011        0    731.0        C/AU   C/H4                           NaN   
4   2011       22   4471.0        C/AU   C/H4                      0.000574   

   (Avg_Weibull_Failure, 10370)  (Avg_Weibull_Failure, 10655)  \
0                           0.0                      0.000211   
1                           0.0                      0.000286   
2                           0.0                      0.000211   
3                           NaN                           NaN   
4                           0.0                      0.000646   

   (Avg_Weibull_Failure, 10C679)  (Avg_Weibull_Failure, 11002)  ...  \
0                            0.0                           0.0  ...   
1                            0.0                           0.0  ...   
2                            0.0                           0.0  ...   
3                            NaN                           NaN  ...   
4                            0.0                           0.0  ...   

   (Avg_Weibull_Failure, 6068)  (Avg_Weibull_Failure, 65)  \
0                     0.000368                        0.0   
1                     0.000450                        0.0   
2                     0.000368                        0.0   
3                          NaN                        NaN   
4                     0.000769                        0.0   

   (Avg_Weibull_Failure, 67R)  (Avg_Weibull_Failure, 7210)  \
0                         0.0                          0.0   
1                         0.0                          0.0   
2                         0.0                          0.0   
3                         NaN                          NaN   
4                         0.0                          0.0   

   (Avg_Weibull_Failure, 7251)  (Avg_Weibull_Failure, 7B155)  \
0                          0.0                           0.0   
1                          0.0                           0.0   
2                          0.0                           0.0   
3                          NaN                           NaN   
4                          0.0                           0.0   

   (Avg_Weibull_Failure, 90)  (Avg_Weibull_Failure, 90T5)  \
0                        0.0                          0.0   
1                        0.0                          0.0   
2                        0.0                          0.0   
3                        NaN                          NaN   
4                        0.0                          0.0   

   (Avg_Weibull_Failure, 94RH7)  (Avg_Weibull_Failure, 96R)  
0                           0.0                    0.004043  
1                           0.0                    0.005769  
2                           0.0                    0.004043  
3                           NaN                         NaN  
4                           0.0                    0.014901  

[5 rows x 116 columns]

### repeat the steps of creating dummy variables and train with xgb

In [43]:
## convert to one-hot encoding
df_X_dummy_1 = pd.get_dummies(df1)

In [44]:
df_X_dummy_1.head(5)

tis_wsd    milge  (Avg_Weibull_Failure, 10346)  \
0       13  14776.0                      0.000363   
1       15  15146.0                      0.000411   
2       13  11198.0                      0.000363   
3        0    731.0                           NaN   
4       22   4471.0                      0.000574   

   (Avg_Weibull_Failure, 10370)  (Avg_Weibull_Failure, 10655)  \
0                           0.0                      0.000211   
1                           0.0                      0.000286   
2                           0.0                      0.000211   
3                           NaN                           NaN   
4                           0.0                      0.000646   

   (Avg_Weibull_Failure, 10C679)  (Avg_Weibull_Failure, 11002)  \
0                            0.0                           0.0   
1                            0.0                           0.0   
2                            0.0                           0.0   
3                            NaN                           NaN   
4                            0.0                           0.0   

   (Avg_Weibull_Failure, 1104)  (Avg_Weibull_Failure, 1109)  \
0                     0.000198                          0.0   
1                     0.000238                          0.0   
2                     0.000198                          0.0   
3                          NaN                          NaN   
4                     0.000388                          0.0   

   (Avg_Weibull_Failure, 1131)  ...  eng_cd_T/UR  eng_cd_T/VN  eng_cd_T/VZ  \
0                          0.0  ...            0            0            0   
1                          0.0  ...            0            0            0   
2                          0.0  ...            0            0            0   
3                          NaN  ...            0            0            0   
4                          0.0  ...            0            0            0   

   eng_cd_T/WA  eng_cd_T/WQ  eng_cd_T/WX  eng_cd_T/YE  eng_cd_T/YF  \
0            0            0            0            0            0   
1            0            0            0            0            0   
2            0            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   eng_cd_T/YG  eng_cd_T/YH  
0            0            0  
1            0            0  
2            0            0  
3            0            0  
4            0            0  

[5 rows x 273 columns]

In [45]:
X_train, X_test, y_train, y_test = train_test_split(df_X_dummy_1, label_encoded_y, test_size = 0.2, random_state = 11)

In [46]:
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

In [47]:
eval_set = [(dtrain, 'train'), (dtest, 'eval')]

In [48]:
gbm_1 = xgb.train(params, dtrain, num_rounds, evals=eval_set, early_stopping_rounds=early_stopping_rounds)

[0]	train-merror:0.772413	eval-merror:0.77581
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0.758066	eval-merror:0.762737
[2]	train-merror:0.754054	eval-merror:0.759022
[3]	train-merror:0.751508	eval-merror:0.756229
[4]	train-merror:0.750632	eval-merror:0.755726
[5]	train-merror:0.750073	eval-merror:0.754693
[6]	train-merror:0.748188	eval-merror:0.752458
[7]	train-merror:0.747629	eval-merror:0.752053
[8]	train-merror:0.746966	eval-merror:0.751271
[9]	train-merror:0.746711	eval-merror:0.751453
[10]	train-merror:0.746316	eval-merror:0.75074
[11]	train-merror:0.745786	eval-merror:0.750615
[12]	train-merror:0.745087	eval-merror:0.750182
[13]	train-merror:0.744588	eval-merror:0.749665
[14]	train-merror:0.744221	eval-merror:0.749246
[15]	train-merror:0.743352	eval-merror:0.748729
[16]	train-merror:0.74295	eval-merror:0.748408
[17]	train-merror:0.742584	eval-merror:0.747779
[1

In [49]:
probs_1 = gbm_1.predict(xgb.DMatrix(X_test))

In [50]:
df_prob_1 = pd.DataFrame(probs_1, columns=le.classes_)

In [51]:
# GET TOP K PREDICTIONS BY PROB - note these are just index
best_n = np.argsort(probs_1, axis=1)[:,-k:]

# GET CATEGORY OF PREDICTIONS
preds_1=[[le.classes_[predicted_cat] for predicted_cat in prediction] for prediction in best_n]

preds_1=[item[::-1] for item in preds_1]

df_top3_1 = pd.DataFrame(preds_1, columns=("Top1", "Top2", "Top3"))

#### 1% accuracy increase

In [52]:
n_correct = 0
for i in range(N):
    if (df_top3_1.iloc[i, 0]==y_test_part[i]) or (df_top3_1.iloc[i, 1]==y_test_part[i]) or (df_top3_1.iloc[i, 2]==y_test_part[i]):
        n_correct += 1
        
print(n_correct/N)

0.46099162011173184


### include text tf-idf fields

In [53]:
lookup_table = 'C:/Users/lguerra5/Documents/FNOP/data/Misspellings_Correction_Synonyms_Lookup_Table_Master.xlsx'

In [55]:
syn_df=pd.read_excel(lookup_table, engine='openpyxl')

In [56]:
syn_df = syn_df[syn_df['Review'] != "D"]

In [57]:
syn_dict=dict(zip(syn_df.misspelled, syn_df.corrected))

In [58]:
def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J":'a',
                       "N":'n',
                       "V":'v',
                       "R":'r'}
    words_and_tags = [(w, tag_dict.get(pos[0],'n')) for w, pos in sent.tags]
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

In [59]:
def TextPreprocess(text):
    # Avoid mixed data type errors and apply str on all rows

    text = str(text).lower()

    #Remove HTML Tags
    text = re.sub('<[^>]*>', '', text)
    r1 = re.compile('<.*?>')
    text = re.sub(r1, ' ', text)
    # Remove any token with numbers and/or special characters

    r2 = re.compile(r'^[0-9\W\_]+$')
    text = re.sub(r2, ' ', text)

    #Removing Punctuations
    text = re.sub(r'[?|!|\'|"|#]',r'',text)
    text = re.sub(r'[.|,|)|(|\|/]',r' ',text)        

    # Tokenize all text data using nltk package

    text = word_tokenize(text)

    # Remove stop words
    stop = stopwords.words('english')
    stop1 = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','c/s','c/states','c/stated','cu', 'cs','c-s', 'cus','tomer', 'stomer', 'tate', 'cust',  'cust states', 'advise','nana','NaN','customer','states','rr','nan','wont','ck','check','pls','please', 'client','owner']

    # Combine default nltk stop list and custom stop list to generate a deduplicated list
    english_stops = list(set(stop+stop1))

    text = [item for item in text if item not in english_stops]

    # Remove special characters occuring as prefixes
    text = [re.sub(r'^[\W\_]+','',str(i)) for i in text]
    # Remove special characters occuring as suffixes

    text = [re.sub(r'[\W\_]+$','',str(i)) for i in text]

    # Replace special characters with spaces when occuring within the token 
    #(i.e. not at the beginning or the end) only for tokens greater than 5 characters in length

    text =[re.sub(r'[\W\_]+', ' ', str(i)) if len(str(i)) > 5 else i for i in text]

    # Replace special characters with null when occuring within the token 
    #(i.e. not at the beginning or the end) only for tokens less than 6 characters in length

    text = [re.sub(r'[\W\_]+', '', str(i)) if len(str(i)) < 6 else i for i in text]
    # Remove stop words after data cleansing steps to remove any residual stop words

    text = [item for item in text if item not in english_stops]

    # remove any numbers after data cleansing steps to remove any residual numeric values

    r3 = re.compile(r'^[0-9\W\_]+$')
    text = [i for i in text if not r3.match(i)]

    #Perform synonym lookup to find matching correctly spelled tokens for misspelled tokens
    #spell=SpellChecker() 

    text = [syn_dict.get(str(i), str(i)) for i in text]

    # Convert the list to string before passing to lemmatization module
    text = ' '.join(text)

    # Perform Lemmatization on the cleaned up data before tfidf vectorization

    text = lemmatize_with_postag(text)

    return(text)

In [63]:
# Test Text Processing Function

In [64]:
sample_txt = df_tmp['cust_txt'][0]

In [65]:
print(sample_txt)

CS NOISE ENGINE AREA   POSSIBLE ALTERNATOR


In [66]:
print(TextPreprocess(sample_txt))

noise engine area possible alternator


In [67]:
text_df = df_tmp[['cust_txt']]

In [68]:
text_df['process_txt']=text_df['cust_txt'].apply(lambda x: TextPreprocess(x))

C:\Users\lguerra5\AppData\Local\Continuum\anaconda3\envs\fnop\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [69]:
text_df[['process_txt']].to_csv("C:/Users/lguerra5/Documents/FNOP/data/processed_cust_txt.csv", index=False)

In [70]:
text_input_df = pd.read_csv("C:/Users/lguerra5/Documents/FNOP/data/processed_cust_txt.csv")

In [71]:
text_input_df.head(4)

process_txt
0              noise engine area possible alternator
1  whine noise engine area ken verify drop last n...
2         battery light come drive whole way morning
3              tow battery light light radio driving

In [72]:
### convert null values of customer_txt into ' '
text_input_df.loc[text_input_df['process_txt'].isnull()==True,'process_txt'] = ' '

In [73]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df = 0.5, min_df = 0.0005)

In [74]:
tfidf_matrix = vectorizer.fit_transform(text_input_df['process_txt'])

In [75]:
tfidf_matrix.shape

(358000, 2076)

In [76]:
tfidf_dense = tfidf_matrix.todense()

In [77]:
tfidf_features = vectorizer.get_feature_names()

In [78]:
tfidf_features[:10]

['1st',
 '2mm',
 '2nd',
 '2nd key',
 '30mph',
 '3mm',
 '3rd',
 '40mph',
 '45mph',
 '4wd']

In [79]:
tfidf_dense[0,:10]

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [80]:
tfidf_df = pd.DataFrame(tfidf_dense, columns = tfidf_features)

In [81]:
tfidf_df.head(5)

1st  2mm  2nd  2nd key  30mph  3mm  3rd  40mph  45mph  4wd  ...  working  \
0  0.0  0.0  0.0      0.0    0.0  0.0  0.0    0.0    0.0  0.0  ...      0.0   
1  0.0  0.0  0.0      0.0    0.0  0.0  0.0    0.0    0.0  0.0  ...      0.0   
2  0.0  0.0  0.0      0.0    0.0  0.0  0.0    0.0    0.0  0.0  ...      0.0   
3  0.0  0.0  0.0      0.0    0.0  0.0  0.0    0.0    0.0  0.0  ...      0.0   
4  0.0  0.0  0.0      0.0    0.0  0.0  0.0    0.0    0.0  0.0  ...      0.0   

   worn  worse  would  would like  would start  wrench  wrench light  yellow  \
0   0.0    0.0    0.0         0.0          0.0     0.0           0.0     0.0   
1   0.0    0.0    0.0         0.0          0.0     0.0           0.0     0.0   
2   0.0    0.0    0.0         0.0          0.0     0.0           0.0     0.0   
3   0.0    0.0    0.0         0.0          0.0     0.0           0.0     0.0   
4   0.0    0.0    0.0         0.0          0.0     0.0           0.0     0.0   

   yesterday  
0        0.0  
1        0.0  
2        0.0  
3        0.0  
4        0.0  

[5 rows x 2076 columns]

### combine tfidf_df with df_X_dummy_1

In [82]:
df_X_dummy_2 = pd.concat([df_X_dummy_1, tfidf_df], axis=1)

In [83]:
df_X_dummy_2.shape

(358000, 2349)

In [84]:
df_X_dummy_2.head(5)

tis_wsd    milge  (Avg_Weibull_Failure, 10346)  \
0       13  14776.0                      0.000363   
1       15  15146.0                      0.000411   
2       13  11198.0                      0.000363   
3        0    731.0                           NaN   
4       22   4471.0                      0.000574   

   (Avg_Weibull_Failure, 10370)  (Avg_Weibull_Failure, 10655)  \
0                           0.0                      0.000211   
1                           0.0                      0.000286   
2                           0.0                      0.000211   
3                           NaN                           NaN   
4                           0.0                      0.000646   

   (Avg_Weibull_Failure, 10C679)  (Avg_Weibull_Failure, 11002)  \
0                            0.0                           0.0   
1                            0.0                           0.0   
2                            0.0                           0.0   
3                            NaN                           NaN   
4                            0.0                           0.0   

   (Avg_Weibull_Failure, 1104)  (Avg_Weibull_Failure, 1109)  \
0                     0.000198                          0.0   
1                     0.000238                          0.0   
2                     0.000198                          0.0   
3                          NaN                          NaN   
4                     0.000388                          0.0   

   (Avg_Weibull_Failure, 1131)  ...  working  worn  worse  would  would like  \
0                          0.0  ...      0.0   0.0    0.0    0.0         0.0   
1                          0.0  ...      0.0   0.0    0.0    0.0         0.0   
2                          0.0  ...      0.0   0.0    0.0    0.0         0.0   
3                          NaN  ...      0.0   0.0    0.0    0.0         0.0   
4                          0.0  ...      0.0   0.0    0.0    0.0         0.0   

   would start  wrench  wrench light  yellow  yesterday  
0          0.0     0.0           0.0     0.0        0.0  
1          0.0     0.0           0.0     0.0        0.0  
2          0.0     0.0           0.0     0.0        0.0  
3          0.0     0.0           0.0     0.0        0.0  
4          0.0     0.0           0.0     0.0        0.0  

[5 rows x 2349 columns]

In [85]:
X_train, X_test, y_train, y_test = train_test_split(df_X_dummy_2, label_encoded_y, test_size = 0.2, random_state = 11)

In [86]:
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

In [87]:
eval_set = [(dtrain, 'train'), (dtest, 'eval')]

In [218]:
gbm_2 = xgb.train(params, dtrain, num_rounds, evals=eval_set, early_stopping_rounds=early_stopping_rounds)

[0]	train-mlogloss:2.34794	eval-mlogloss:2.36259
[1]	train-mlogloss:2.15791	eval-mlogloss:2.17639
[2]	train-mlogloss:2.03335	eval-mlogloss:2.05468
[3]	train-mlogloss:1.93273	eval-mlogloss:1.95629
[4]	train-mlogloss:1.84919	eval-mlogloss:1.87539
[5]	train-mlogloss:1.77738	eval-mlogloss:1.80576
[6]	train-mlogloss:1.71371	eval-mlogloss:1.74361
[7]	train-mlogloss:1.65722	eval-mlogloss:1.68860
[8]	train-mlogloss:1.60613	eval-mlogloss:1.63899
[9]	train-mlogloss:1.55957	eval-mlogloss:1.59411
[10]	train-mlogloss:1.51774	eval-mlogloss:1.55343
[11]	train-mlogloss:1.47948	eval-mlogloss:1.51687
[12]	train-mlogloss:1.44422	eval-mlogloss:1.48304
[13]	train-mlogloss:1.41094	eval-mlogloss:1.45112
[14]	train-mlogloss:1.38149	eval-mlogloss:1.42268
[15]	train-mlogloss:1.35361	eval-mlogloss:1.39613
[16]	train-mlogloss:1.32779	eval-mlogloss:1.37157
[17]	train-mlogloss:1.30320	eval-mlogloss:1.34817
[18]	train-mlogloss:1.28059	eval-mlogloss:1.32684
[19]	train-mlogloss:1.25954	eval-mlogloss:1.30706
[20]	train

In [219]:
probs_2 = gbm_2.predict(xgb.DMatrix(X_test))

In [220]:
df_prob_2 = pd.DataFrame(probs_2, columns=le.classes_)

In [221]:
# GET TOP K PREDICTIONS BY PROB - note these are just index
best_n = np.argsort(probs_2, axis=1)[:,-k:]

# GET CATEGORY OF PREDICTIONS
preds_2=[[le.classes_[predicted_cat] for predicted_cat in prediction] for prediction in best_n]

preds_2=[item[::-1] for item in preds_2]

df_top3_2 = pd.DataFrame(preds_2, columns=("Top1", "Top2", "Top3"))

In [222]:
n_correct = 0
for i in range(N):
    if (df_top3_2.iloc[i, 0]==y_test_part[i]) or (df_top3_2.iloc[i, 1]==y_test_part[i]) or (df_top3_2.iloc[i, 2]==y_test_part[i]):
        n_correct += 1
        
print(n_correct/N)

0.8891480446927374
